## 1. Import Libraries and Setup

In [1]:
# Setup Python path to find project modules
import sys
import os
from pathlib import Path

# Get the project root (Machine Learning directory)
project_root = Path.cwd().parent.parent if "src/cle/extract" in str(Path.cwd()) else Path.cwd()
if project_root.name != "Machine Learning":
    # If we're not in the expected structure, try to find Machine Learning directory
    current = Path.cwd()
    while current != current.parent:
        if current.name == "Machine Learning":
            project_root = current
            break
        current = current.parent

print(f"Project root: {project_root}")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added {project_root} to Python path")

# Change working directory to project root for relative paths to work
if os.getcwd() != str(project_root):
    os.chdir(str(project_root))
    print(f"Changed working directory to: {os.getcwd()}")

print("Python path setup complete!")

Project root: /Users/sam/Desktop/FYP/Machine Learning
Added /Users/sam/Desktop/FYP/Machine Learning to Python path
Changed working directory to: /Users/sam/Desktop/FYP/Machine Learning
Python path setup complete!


In [2]:
import sys
from pathlib import Path
from typing import Dict, List, Optional, Set

import pandas as pd
from tqdm import tqdm

from src.cle.config import load_config
from src.cle.data.load_data import TASK_FILTER, load_load_index
from src.cle.extract.features import compute_window_features, get_feature_names
from src.cle.extract.landmarks import FaceMeshExtractor
from src.cle.extract.per_frame import extract_frame_features
from src.cle.extract.windowing import (
    interpolate_gaps,
    sliding_window_indices,
    validate_window_quality,
)
from src.cle.logging_setup import get_logger, setup_logging
from src.cle.utils.io import open_video, save_features_csv
from src.cle.utils.timers import timer

# Setup logging
setup_logging(
    level="INFO",
    log_dir="logs",
    log_file="pipeline_loadindex.log",
)
logger = get_logger(__name__)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration Parameters

Set the pipeline parameters here:

In [3]:
# Pipeline Configuration
VIDEOS_DIR = "/Users/sam/Desktop/FYP/Videos"  # Path to Videos directory containing subject folders
LOAD_INDEX_FILE = "/Users/sam/Desktop/FYP/Machine Learning/self_assessments_loadindex.csv"  # Path to load index CSV
OUTPUT_FILE = "data/processed/loadindex_features.csv"  # Output path for features CSV
CONFIG_FILE = "configs/regression.yaml"  # Configuration file path

# Tasks to process
TASKS_TO_PROCESS = ["Math", "Counting1", "Counting2", "Speaking"]

# Optional: Specific subjects to process (None = all subjects)
SPECIFIC_SUBJECTS = None  # e.g., ["2ea4", "2hpu"] or None for all

print(f"Videos directory: {VIDEOS_DIR}")
print(f"Load index file: {LOAD_INDEX_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Tasks to process: {TASKS_TO_PROCESS}")
print(f"Specific subjects: {SPECIFIC_SUBJECTS or 'All subjects'}")

Videos directory: /Users/sam/Desktop/FYP/Videos
Load index file: /Users/sam/Desktop/FYP/Machine Learning/self_assessments_loadindex.csv
Output file: data/processed/loadindex_features.csv
Tasks to process: ['Math', 'Counting1', 'Counting2', 'Speaking']
Specific subjects: All subjects


## 3. Helper Functions

In [12]:
def discover_videos(
    videos_dir: Path,
    tasks: Set[str],
    load_index_subjects: Set[str],
) -> List[Dict]:
    """
    Discover all videos matching the specified tasks and subjects.

    Args:
        videos_dir: Path to Videos directory containing subject folders
        tasks: Set of task names to include
        load_index_subjects: Set of subjects that have load index entries

    Returns:
        List of video info dictionaries with keys: subject, task, video_path
    """
    videos = []

    # Iterate through subject directories
    for subject_dir in sorted(videos_dir.iterdir()):
        if not subject_dir.is_dir():
            continue

        subject_id = subject_dir.name

        # Skip subjects not in load index
        if subject_id not in load_index_subjects:
            logger.debug(f"Skipping subject {subject_id} - not in load index")
            continue

        # Find videos for specified tasks
        for task in tasks:
            video_name = f"{subject_id}_{task}.mp4"
            video_path = subject_dir / video_name

            if video_path.exists():
                videos.append({
                    "subject": subject_id,
                    "task": task,
                    "video_path": str(video_path),
                })
            else:
                logger.warning(f"Video not found: {video_path}")

    return videos

In [13]:
def process_video(
    video_path: str,
    config: Dict,
    extractor: FaceMeshExtractor,
) -> List[Dict]:
    """
    Process a single video and extract per-frame features.

    Args:
        video_path: Path to video file
        config: Configuration dictionary
        extractor: FaceMeshExtractor instance

    Returns:
        List of per-frame feature dictionaries
    """
    cap, vid_meta = open_video(video_path)
    fps = vid_meta["fps"] if vid_meta["fps"] > 0 else config.get("fps_fallback", 30.0)

    frame_features = []
    frame_idx = 0

    with tqdm(
        total=vid_meta["frame_count"],
        desc=f"  {Path(video_path).name}",
        leave=False,
    ) as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Extract landmarks
            landmark_result = extractor.process_frame(frame)

            # Extract per-frame features
            features = extract_frame_features(frame, landmark_result)
            features["frame_idx"] = frame_idx
            features["timestamp_s"] = frame_idx / fps

            frame_features.append(features)
            frame_idx += 1
            pbar.update(1)

    cap.release()

    # Interpolate only single-frame dropouts (avoid creating fake blinks)
    frame_features = interpolate_gaps(frame_features, max_gap=1)

    valid_count = len([f for f in frame_features if f["valid"]])
    logger.debug(
        f"Processed {len(frame_features)} frames ({valid_count} valid)"
    )

    return frame_features

In [14]:
def extract_windows(
    frame_features: List[Dict],
    fps: float,
    config: Dict,
    video_info: Dict,
    load_0_1: float,
) -> List[Dict]:
    """
    Extract window-level features from per-frame features.

    Args:
        frame_features: List of per-frame feature dictionaries
        fps: Frames per second
        config: Configuration dictionary
        video_info: Video info with subject, task, video_path
        load_0_1: Continuous load label from load index

    Returns:
        List of window feature dictionaries
    """
    # Generate sliding windows
    windows = sliding_window_indices(
        n_frames=len(frame_features),
        fps=fps,
        length_s=config.get("windows.length_s", 10.0),
        step_s=config.get("windows.step_s", 2.5),
    )

    window_features_list = []

    for start_idx, end_idx, start_time_s, end_time_s in windows:
        # Extract window data
        window_data = frame_features[start_idx:end_idx]

        # Validate window quality
        is_valid, bad_ratio = validate_window_quality(
            window_data,
            max_bad_ratio=config.get("quality.max_bad_frame_ratio", 0.05),
        )

        if not is_valid:
            logger.debug(
                f"Skipping window [{start_time_s:.1f}s - {end_time_s:.1f}s] "
                f"(bad_ratio={bad_ratio:.2f})"
            )
            continue

        # Compute window features
        window_features = compute_window_features(window_data, config, fps)

        # Add metadata with load index label
        window_features.update({
            "t_start_s": start_time_s,
            "t_end_s": end_time_s,
            "user_id": video_info["subject"],
            "task": video_info["task"],
            "video": video_info["video_path"],
            "load_0_1": load_0_1,
            "role": "train",  # Default role, can be changed later
        })

        window_features_list.append(window_features)

    return window_features_list

## 4. Load Configuration and Data

In [5]:
print("="*80)
print("Starting LOAD INDEX feature extraction pipeline")
print("="*80)

# Load configuration
config = load_config(CONFIG_FILE)
print(f"Loaded configuration from {CONFIG_FILE}")

# Determine tasks to extract
tasks = set(TASKS_TO_PROCESS) if TASKS_TO_PROCESS else TASK_FILTER
print(f"Tasks to extract: {tasks}")

# Load load index
load_index_path = Path(LOAD_INDEX_FILE)
if not load_index_path.exists():
    raise FileNotFoundError(f"Load index file not found: {load_index_path}")

load_df = load_load_index(str(load_index_path))
print(f"Loaded load index with {len(load_df)} entries")

# Filter load index to specified tasks
load_df = load_df[load_df["task"].isin(tasks)]
print(f"Load index entries for specified tasks: {len(load_df)}")

# Display first few rows
print("\nFirst 5 rows of load index:")
display(load_df.head())

Starting LOAD INDEX feature extraction pipeline
Loaded configuration from configs/regression.yaml
Tasks to extract: {'Speaking', 'Counting1', 'Math', 'Counting2'}
2025-12-08 18:10:13,350 - cle.src.cle.data.load_data - INFO - Loaded load index: 714 rows, 65 subjects, 11 tasks
2025-12-08 18:10:13,351 - cle.src.cle.data.load_data - INFO - load_0_1 range: [0.000, 1.000]
2025-12-08 18:10:13,351 - cle.src.cle.data.load_data - INFO - load_0_1 range: [0.000, 1.000]
Loaded load index with 714 entries
Load index entries for specified tasks: 260

First 5 rows of load index:
Loaded load index with 714 entries
Load index entries for specified tasks: 260

First 5 rows of load index:


,subject,task,stress,relax,load_0_1
1,2ea4,Counting1,7.0,4.0,0.65
2,2ea4,Counting2,6.0,5.0,0.55
4,2ea4,Math,6.0,6.0,0.50
7,2ea4,Speaking,6.0,4.0,0.60
12,2hpu,Counting1,7.0,4.0,0.65


In [6]:
# Get subjects from load index
load_index_subjects = set(load_df["subject"].unique())

# Filter to specific subjects if requested
if SPECIFIC_SUBJECTS:
    requested_subjects = set(SPECIFIC_SUBJECTS)
    load_index_subjects = load_index_subjects & requested_subjects
    print(f"Filtering to requested subjects: {load_index_subjects}")

print(f"Subjects with load index data: {len(load_index_subjects)}")
print(f"Subject list: {sorted(list(load_index_subjects))}")

# Create lookup for load_0_1 values
load_lookup = {}
for _, row in load_df.iterrows():
    key = (row["subject"], row["task"])
    load_lookup[key] = row["load_0_1"]

print(f"Created load lookup with {len(load_lookup)} entries")

Subjects with load index data: 65
Subject list: ['2ea4', '2hpu', '2z7d', '37ir', '45lx', '4e8r', '4woj', '5f7t', '6g6y', '6k5f', '71i5', '7h5u', '7m3c', '8g4y', '8i4i', '9j3o', '9t6n', '9txq', 'a1k9', 'b2l8', 'b9w0', 'bfl5', 'c3m7', 'chdf', 'ctzy', 'cxj0', 'd4n6', 'dmbd', 'e5p4', 'f6q3', 'g7r2', 'g9j5', 'h7j3', 'h8r2', 'h8s1', 'hh2e', 'hvpa', 'i9t9', 'iqyg', 'j1u8', 'j9h8', 'k2v7', 'k67g', 'kkf5', 'kycf', 'm8g5', 'p9i3', 'ql3b', 'qw5t', 'qx2o', 'r0a3', 'r3zm', 'r5s8', 't6v9', 'tmvd', 'u3v9', 'uymz', 'uyrl', 'v8mh', 'w2t5', 'wfsl', 'wssm', 'x1q3', 'y8c3', 'y9z6']
Created load lookup with 260 entries


## 5. Discover Videos

In [7]:
# Discover videos
videos_dir = Path(VIDEOS_DIR)
if not videos_dir.exists():
    raise FileNotFoundError(f"Videos directory not found: {videos_dir}")

videos = discover_videos(videos_dir, tasks, load_index_subjects)
print(f"Found {len(videos)} videos to process")

if not videos:
    raise ValueError("No videos found to process")

# Show summary before processing
subjects_found = set(v["subject"] for v in videos)
tasks_found = set(v["task"] for v in videos)
print(f"Subjects found: {len(subjects_found)} - {sorted(subjects_found)}")
print(f"Tasks found: {sorted(tasks_found)}")

# Show video breakdown by task and subject
video_df = pd.DataFrame(videos)
print("\nVideo count by task:")
display(video_df.groupby('task').size())
print("\nVideo count by subject:")
display(video_df.groupby('subject').size().head(10))  # Show first 10 subjects

Found 212 videos to process
Subjects found: 53 - ['2ea4', '2hpu', '2z7d', '45lx', '4e8r', '4woj', '5f7t', '6g6y', '6k5f', '71i5', '7h5u', '8g4y', '8i4i', '9j3o', '9t6n', '9txq', 'a1k9', 'b2l8', 'b9w0', 'bfl5', 'c3m7', 'chdf', 'ctzy', 'cxj0', 'd4n6', 'e5p4', 'f6q3', 'g7r2', 'g9j5', 'h7j3', 'h8r2', 'h8s1', 'i9t9', 'iqyg', 'j9h8', 'k2v7', 'k67g', 'kkf5', 'kycf', 'm8g5', 'p9i3', 'qw5t', 'r3zm', 'r5s8', 't6v9', 'tmvd', 'uymz', 'v8mh', 'w2t5', 'wssm', 'x1q3', 'y8c3', 'y9z6']
Tasks found: ['Counting1', 'Counting2', 'Math', 'Speaking']

Video count by task:


task
Counting1    53
Counting2    53
Math         53
Speaking     53
dtype: int64


Video count by subject:


subject
2ea4    4
2hpu    4
2z7d    4
45lx    4
4e8r    4
4woj    4
5f7t    4
6g6y    4
6k5f    4
71i5    4
dtype: int64

## 6. Process Videos and Extract Features

This is the main processing loop. It may take a while depending on the number of videos.

In [10]:
# Initialize variables
all_window_features = []
processed_count = 0
error_count = 0

print("\nStarting video processing...")
print(f"Total videos to process: {len(videos)}")
print("This may take a while depending on the number and length of videos.")


Starting video processing...
Total videos to process: 212
This may take a while depending on the number and length of videos.


In [15]:
# Process videos with face mesh extractor
with FaceMeshExtractor(
    min_detection_confidence=config.get("quality.min_face_conf", 0.5),
    refine_landmarks=True,
) as extractor:

    for video_info in tqdm(videos, desc="Processing videos"):
        subject = video_info["subject"]
        task = video_info["task"]
        video_path = video_info["video_path"]

        # Get load_0_1 value
        load_key = (subject, task)
        if load_key not in load_lookup:
            print(f"WARNING: No load index for {load_key}, skipping")
            continue

        load_0_1 = load_lookup[load_key]

        print(f"\nProcessing {subject}/{task} (load_0_1={load_0_1:.2f})")

        try:
            with timer(f"{subject}_{task}", log=False):
                # Process video to get per-frame features
                frame_features = process_video(
                    video_path,
                    config.to_dict(),
                    extractor,
                )

                if not frame_features:
                    print(f"WARNING: No frames extracted from {video_path}")
                    error_count += 1
                    continue

                # Get FPS
                cap, vid_meta = open_video(video_path)
                fps = (
                    vid_meta["fps"]
                    if vid_meta["fps"] > 0
                    else config.get("fps_fallback", 30.0)
                )
                cap.release()

                # Extract window features
                window_features = extract_windows(
                    frame_features,
                    fps,
                    config.to_dict(),
                    video_info,
                    load_0_1,
                )

                if window_features:
                    all_window_features.extend(window_features)
                    processed_count += 1
                    print(f"  ✓ Extracted {len(window_features)} windows")
                else:
                    print(f"  ✗ No valid windows extracted")
                    error_count += 1

        except Exception as e:
            print(f"ERROR processing {video_path}: {e}")
            error_count += 1
            continue

        # Print progress every 10 videos
        if (processed_count + error_count) % 10 == 0:
            print(f"\nProgress: {processed_count + error_count}/{len(videos)} videos processed")
            print(f"Successful: {processed_count}, Errors: {error_count}")

print(f"\nVideo processing complete!")
print(f"Successfully processed: {processed_count} videos")
print(f"Errors: {error_count} videos")
print(f"Total windows extracted: {len(all_window_features)}")

2025-12-08 18:12:07,466 - cle.src.cle.extract.landmarks - INFO - Initialized FaceMeshExtractor (refine_landmarks=True, min_conf=0.5)


I0000 00:00:1765217527.466604 57536202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
W0000 00:00:1765217527.468435 57541105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing videos:   0%|          | 0/212 [00:00<?, ?it/s]


Processing 2ea4/Speaking (load_0_1=0.60)
2025-12-08 18:12:07,482 - cle.src.cle.utils.io - INFO - Opened video: 2ea4_Speaking.mp4 (1280x720, 5.0 fps, 300 frames)


W0000 00:00:1765217527.478225 57541101 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765217527.499065 57541106 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
W0000 00:00:1765217527.499065 57541106 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


2025-12-08 18:12:08,938 - cle.src.cle.utils.io - INFO - Opened video: 2ea4_Speaking.mp4 (1280x720, 5.0 fps, 300 frames)


...

  ✓ Extracted 21 windows

Processing y9z6/Counting2 (load_0_1=0.60)
2025-12-08 18:24:09,215 - cle.src.cle.utils.io - INFO - Opened video: y9z6_Counting2.mp4 (1280x720, 5.0 fps, 300 frames)


  y9z6_Counting2.mp4:   0%|          | 0/300 [00:00<?, ?it/s]

2025-12-08 18:24:11,078 - cle.src.cle.utils.io - INFO - Opened video: y9z6_Counting2.mp4 (1280x720, 5.0 fps, 300 frames)


Processing videos: 100%|██████████| 212/212 [12:03<00:00,  3.41s/it]

  ✓ Extracted 21 windows

Video processing complete!
Successfully processed: 212 videos
Errors: 212 videos
Total windows extracted: 4415


## 7. Process and Save Results

In [16]:
# Check if we got any features
if not all_window_features:
    raise ValueError("No features extracted from any video")

# Convert to DataFrame
df = pd.DataFrame(all_window_features)
print(f"Created DataFrame with {len(df)} rows and {len(df.columns)} columns")

# Show basic info about the DataFrame
print("\nDataFrame info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)[:10]}...")  # Show first 10 column names
print("\nFirst few rows:")
display(df[['user_id', 'task', 'load_0_1', 't_start_s', 't_end_s']].head())

Created DataFrame with 4415 rows and 16 columns

DataFrame info:
Shape: (4415, 16)
Columns: ['blink_rate', 'blink_count', 'mean_blink_duration', 'ear_std', 'mean_brightness', 'std_brightness', 'perclos', 'mean_quality', 'valid_frame_ratio', 't_start_s']...

First few rows:


,user_id,task,load_0_1,t_start_s,t_end_s
0,2ea4,Speaking,0.6,0.0,10.0
1,2ea4,Speaking,0.6,2.4,12.4
2,2ea4,Speaking,0.6,4.8,14.8
3,2ea4,Speaking,0.6,7.2,17.2
4,2ea4,Speaking,0.6,9.6,19.6


In [17]:
# Reorder columns: metadata first, then features
feature_names = get_feature_names(config.to_dict())
metadata_cols = [
    "user_id",
    "task", 
    "video",
    "load_0_1",
    "role",
    "t_start_s",
    "t_end_s",
]
ordered_cols = metadata_cols + feature_names

# Ensure all columns exist
for col in ordered_cols:
    if col not in df.columns:
        print(f"WARNING: Missing column {col}, adding with zeros")
        df[col] = 0.0

df = df[ordered_cols]
print(f"Reordered DataFrame to {len(ordered_cols)} columns")
print(f"Metadata columns: {metadata_cols}")
print(f"Feature columns: {len(feature_names)} features")

Reordered DataFrame to 16 columns
Metadata columns: ['user_id', 'task', 'video', 'load_0_1', 'role', 't_start_s', 't_end_s']
Feature columns: 9 features


In [18]:
# Save features
save_features_csv(df, OUTPUT_FILE, include_index=False)
print(f"Features saved to: {OUTPUT_FILE}")

# Verify file was created
if Path(OUTPUT_FILE).exists():
    file_size = Path(OUTPUT_FILE).stat().st_size / (1024*1024)  # Size in MB
    print(f"File size: {file_size:.2f} MB")
else:
    print("ERROR: Output file was not created!")

2025-12-08 18:48:21,941 - cle.src.cle.utils.io - INFO - Saved 4415 feature rows to data/processed/loadindex_features.csv
Features saved to: data/processed/loadindex_features.csv
File size: 0.73 MB
Features saved to: data/processed/loadindex_features.csv
File size: 0.73 MB


## 8. Summary Statistics

In [19]:
print("="*80)
print("EXTRACTION COMPLETE - SUMMARY STATISTICS")
print("="*80)

print(f"Videos processed successfully: {processed_count}")
print(f"Videos with errors: {error_count}")
print(f"Total windows extracted: {len(df)}")
print(f"Unique subjects: {df['user_id'].nunique()}")
print(f"Unique tasks: {df['task'].nunique()}")

unique_sessions = df.groupby(['user_id', 'task']).ngroups
print(f"Unique sessions: {unique_sessions}")
print(f"Windows per session (mean): {len(df) / unique_sessions:.1f}")

print(f"load_0_1 range: [{df['load_0_1'].min():.3f}, {df['load_0_1'].max():.3f}]")
print(f"Features saved to: {OUTPUT_FILE}")

print("\nTask distribution:")
display(df['task'].value_counts())

print("\nLoad index distribution:")
display(df['load_0_1'].describe())

print("="*80)

EXTRACTION COMPLETE - SUMMARY STATISTICS
Videos processed successfully: 212
Videos with errors: 212
Total windows extracted: 4415
Unique subjects: 53
Unique tasks: 4
Unique sessions: 212
Windows per session (mean): 20.8
load_0_1 range: [0.050, 1.000]
Features saved to: data/processed/loadindex_features.csv

Task distribution:


task
Math         1112
Counting1    1107
Speaking     1105
Counting2    1091
Name: count, dtype: int64


Load index distribution:


count    4415.000000
mean        0.571076
std         0.204371
min         0.050000
25%         0.450000
50%         0.600000
75%         0.700000
max         1.000000
Name: load_0_1, dtype: float64

## 9. Data Quality Checks

In [20]:
# Check for missing values
print("Missing values per column:")
missing_vals = df.isnull().sum()
missing_vals = missing_vals[missing_vals > 0]
if len(missing_vals) > 0:
    display(missing_vals)
else:
    print("No missing values found!")

# Check data types
print("\nData types:")
print(df.dtypes.value_counts())

# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Sample of the final dataset
print("\nSample of final dataset:")
sample_cols = ['user_id', 'task', 'load_0_1', 't_start_s', 't_end_s'] + feature_names[:5]
display(df[sample_cols].sample(min(10, len(df))))

Missing values per column:
No missing values found!

Data types:
float64    12
object      4
Name: count, dtype: int64

Duplicate rows: 0

Sample of final dataset:


,user_id,task,load_0_1,t_start_s,t_end_s,blink_rate,blink_count,mean_blink_duration,ear_std,mean_brightness
928,8g4y,Speaking,0.75,33.600,43.600,18.0,3.0,200.0,0.070998,91.089837
1959,cxj0,Counting1,0.85,38.400,48.400,18.0,3.0,200.0,0.149922,107.814225
3232,kycf,Math,0.50,19.200,29.200,6.0,1.0,200.0,0.046440,105.032109
1815,chdf,Math,0.75,45.600,55.600,0.0,0.0,0.0,0.052405,83.652326
145,2hpu,Math,0.50,47.424,57.424,30.0,5.0,204.8,0.110244,129.982808
605,6g6y,Speaking,0.65,40.800,50.800,6.0,1.0,200.0,0.071761,117.540260
1836,chdf,Counting2,0.80,45.600,55.600,6.0,1.0,200.0,0.074676,78.034116
1206,9t6n,Counting1,0.70,45.600,55.600,0.0,0.0,0.0,0.047595,63.154633
3075,k67g,Math,0.60,45.600,55.600,24.0,4.0,200.0,0.122027,110.577927
500,4woj,Counting2,0.40,42.432,52.432,0.0,0.0,0.0,0.009018,111.556390
